In [1]:
import pickle
import pandas as pd
import numpy as np

/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/pandas/compat/_optional.py:149: UserWarning: Pandas requires version '1.3.1' or newer of 'bottleneck' (version '1.2.1' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# Train/Validation Split
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/train_df.pickle', 'rb') as f:
    train_df = pickle.load(f)
    
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/val_df.pickle', 'rb') as f:
    val_df = pickle.load(f)
    
with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/test_df.pickle', 'rb') as f:
    test_df = pickle.load(f)
    
# CLEANING DATASETS we dont care about wall(2), background(0), outdoor(1), garage (10), undefined (11), railing (8)
train_df = train_df[~train_df['Room'].isin([0, 1, 2, 10, 11, 8])]
train_df = train_df.loc[:, train_df.columns != 'No Icon']

# CLEANING DATASETS we dont care about wall(2), background(0), outdoor(1), garage (10), undefined (11), railing (8)
val_df = val_df[~val_df['Room'].isin([0, 1, 2, 10, 11, 8])]
val_df = val_df.loc[:, val_df.columns != 'No Icon']

# CLEANING DATASETS we dont care about wall(2), background(0), outdoor(1), garage (10), undefined (11), railing (8)
test_df = test_df[~test_df['Room'].isin([0, 1, 2, 10, 11, 8])]
test_df = test_df.loc[:, test_df.columns != 'No Icon']

X_train = train_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]
y_train = train_df['Room']

X_val = val_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]
y_val = val_df['Room']

X_test = test_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]
y_test = test_df['Room']

# Classification Algorithms

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


# Define the MNB classifier model
mnb = MultinomialNB()

# Train the model on the training set
mnb.fit(X_train, y_train)

# Predict the target variable on the val set
y_pred = mnb.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.82      0.96      0.88       358
           4       0.70      0.25      0.36       337
           5       0.36      0.96      0.53       632
           6       1.00      0.81      0.89       556
           7       0.43      0.04      0.08       450
           9       0.71      0.03      0.05       382

    accuracy                           0.56      2715
   macro avg       0.67      0.51      0.47      2715
weighted avg       0.66      0.56      0.49      2715

Accuracy: 0.5576427255985267


In [4]:
# Get the predicted probabilities on the validation set
probs = pd.DataFrame(mnb.predict_proba(X_val), columns=mnb.classes_)

# Combine the actual and predicted values with the probabilities
mnb_predictions = pd.DataFrame({'Actual': y_val, 'Predicted': y_pred}).reset_index(drop=True)
probs = pd.concat([mnb_predictions, probs], axis=1)

# Get the maximum probability for each instance
probs_max = probs.iloc[:, 2:].max(axis=1)
probs_max = pd.concat([probs.iloc[:, 0:2], probs_max], axis=1)
probs_max.columns.values[2] = "probability"

# Vary the probability threshold and compute the resulting accuracy
for i in np.arange(0, 1, 0.01):
    all_rows = probs_max[probs_max['probability'] >= i]
    correct = all_rows.query('Actual == Predicted')
    ratio = len(correct) / len(all_rows)
    if ratio >= 0.9:
        print("Ratio:", ratio)
        print("Threshold:", i)
        break

# Compute the accuracy for the threshold of >=i
accuracy_high = len(probs_max.query(f'probability >= {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability >= {i}'))
print(f"Accuracy for threshold >= {i}:", accuracy_high)

# Compute the accuracy for the threshold of <i
accuracy_low = len(probs_max.query(f'probability < {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability < {i}'))
print(f"Accuracy for threshold < {i}:", accuracy_low)

Ratio: 0.9010204081632653
Threshold: 0.5
Accuracy for threshold >= 0.5: 0.9010204081632653
Accuracy for threshold < 0.5: 0.3636887608069164


In [5]:
from sklearn.ensemble import RandomForestClassifier

# Define the random forest classifier model
rfc = RandomForestClassifier()

# Train the model on the training set
rfc.fit(X_train, y_train)

# Predict the target variable on the validation set
y_pred = rfc.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.96      0.93      0.94       358
           4       0.34      0.88      0.49       337
           5       0.51      0.64      0.57       632
           6       0.92      0.86      0.89       556
           7       0.45      0.16      0.24       450
           9       0.40      0.01      0.02       382

    accuracy                           0.59      2715
   macro avg       0.60      0.58      0.53      2715
weighted avg       0.61      0.59      0.54      2715

Accuracy: 0.5863720073664825


In [6]:
from sklearn.neighbors import KNeighborsClassifier

# Define the k-nearest neighbors classifier model
knn = KNeighborsClassifier()

# Train the model on the training set
knn.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = knn.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.96      0.93      0.95       358
           4       0.81      0.24      0.37       337
           5       0.50      0.66      0.57       632
           6       0.99      0.83      0.90       556
           7       0.42      0.17      0.24       450
           9       0.24      0.51      0.33       382

    accuracy                           0.57      2715
   macro avg       0.65      0.56      0.56      2715
weighted avg       0.65      0.57      0.57      2715

Accuracy: 0.574585635359116


/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [7]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression classifier model
lr = LogisticRegression(max_iter=1000)

# Train the model on the training set
lr.fit(X_train, y_train)

# Predict the target variable on the testing set
y_pred = lr.predict(X_val)

# Evaluate the model performance
print(classification_report(y_val, y_pred))

# Compute the accuracy score
accuracy = accuracy_score(y_val, y_pred)

# Print the accuracy score
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           3       0.96      0.93      0.95       358
           4       0.62      0.28      0.38       337
           5       0.37      0.96      0.53       632
           6       0.97      0.83      0.89       556
           7       0.49      0.08      0.13       450
           9       0.50      0.00      0.01       382

    accuracy                           0.57      2715
   macro avg       0.65      0.51      0.48      2715
weighted avg       0.64      0.57      0.50      2715

Accuracy: 0.565377532228361


In [8]:
# Predict probabilities for each class on the validation set
probs = pd.DataFrame(lr.predict_proba(X_val), columns=lr.classes_)

# Combine the actual and predicted values with the probabilities
lr_predictions = pd.DataFrame({'Actual': y_val, 'Predicted': lr.predict(X_val)}).reset_index(drop=True)
probs = pd.concat([lr_predictions, probs], axis=1)

# Get the maximum probability for each instance
probs_max = probs.iloc[:, 2:].max(axis=1)
probs_max = pd.concat([probs.iloc[:, 0:2], probs_max], axis=1)
probs_max.columns.values[2] = "probability"

# Vary the probability threshold and compute the resulting accuracy
for i in np.arange(0, 1, 0.01):
    all_rows = probs_max[probs_max['probability'] >= i]
    correct = all_rows.query('Actual == Predicted')
    ratio = len(correct) / len(all_rows)
    if ratio >= 0.9:
        print("Ratio:", ratio)
        print("Threshold:", i)
        break

# Compute the accuracy for the threshold of >=i
accuracy_high = len(probs_max.query(f'probability >= {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability >= {i}'))
print(f"Accuracy for threshold >= {i}:", accuracy_high)

# Compute the accuracy for the threshold of <i
accuracy_low = len(probs_max.query(f'probability < {i}').query('Actual == Predicted')) / len(probs_max.query(f'probability < {i}'))
print(f"Accuracy for threshold < {i}:", accuracy_low)

Ratio: 0.9323770491803278
Threshold: 0.46
Accuracy for threshold >= 0.46: 0.9323770491803278
Accuracy for threshold < 0.46: 0.359401955146636


In [9]:
filename = 'lr_classification_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(lr, file)

# Modifying Floorplans

In [10]:
# Define the list of pickle files
files = ['/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/train_df.pickle', 
         '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/test_df.pickle', 
         '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/feature_extraction/val_df.pickle']

# Load the pickle files into a list of dataframes
dfs = []
for file in files:
    with open(file, 'rb') as f:
        df = pickle.load(f)
        dfs.append(df)

# Combine the dataframes into a single dataframe
df = pd.concat(dfs)

# Map the values in the 'Room' column to the room classes
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
df['Room'] = df['Room'].map(dict(enumerate(room_classes)))

# Add a 'Set' column to the dataframe
set_names = ['Train'] * len(dfs[0]) + ['Test'] * len(dfs[1]) + ['Validation'] * len(dfs[2])
df['Set'] = set_names


## Manually rules for undefined rooms

In [11]:
print(f"Number of undefined rooms: {len(df[df['Room'] == 'Undefined'])}")

# Count the number of rooms that are undefined and contain a toilet or sauna bench
num_undefined_toilet_sauna = len(df[(df['Room'] == 'Undefined') & ((df['Toilet']>0) | (df['Sauna Bench']>0))])

# Print the result
print(f"Number of undefined rooms with a toilet or sauna bench: {num_undefined_toilet_sauna}")


# Set the 'Room' value for rows that are undefined and have a toilet or sauna bench to 'Bath Room'
df.loc[(df['Room'] == 'Undefined') & ((df['Toilet'] > 0) | (df['Sauna Bench'] > 0)), 'Room'] = 'Bath'


print(f"Number of undefined rooms after modification: {len(df[df['Room'] == 'Undefined'])}")


Number of undefined rooms: 15750
Number of undefined rooms with a toilet or sauna bench: 4810
Number of undefined rooms after modification: 10940


## Filling with the multiclass logistic regression and probability threshold

In [12]:
# List of values to exclude, because we dont want to predict these rooms. when we care about accuracy, we don't care about predicting rooms of these type to another type, that's why i remove it from val and test as well
exclude = ['Wall', 'Background', 'Outdoor', 'Garage', 'Railing']

# Create a new dataframe with rows where the 'Room' value is not in the exclude list
df = df[~df['Room'].isin(exclude)]
#remove no icon from the columns
df = df.loc[:, df.columns != 'No Icon']

# Get the rows with undefined rooms
undefined_df=df[df['Room']=='Undefined']

# Get the feature matrix for the undefined rooms
X_undefined = undefined_df[['Window', 'Door', 'Closet', 'Electrical Applience', 'Toilet', 'Sink', 'Sauna Bench', 'Fire Place', 'Bathtub', 'Chimney']]

# Predict the probabilities for each class for the undefined rooms
probs_undefined = lr.predict_proba(X_undefined)

# Get the predicted class for each row
predicted_classes = lr.classes_[np.argmax(probs_undefined, axis=1)]

# Get the maximum predicted probability for each row
max_probs = np.max(probs_undefined, axis=1)

# Replace the Room value for rows where the max probability is greater than or equal to the threshold
undefined_copy = undefined_df.copy()
undefined_copy.loc[max_probs >= i, 'Room'] = predicted_classes[max_probs >= i]
undefined_copy.reset_index(drop=True, inplace=True) # reset the index

# List of values to exclude
exclude = ['Wall', 'Background', 'Outdoor', 'Garage', 'Railing']

# Create a new dataframe with rows where the 'Room' value is not in the exclude list
filtered_df = df[~df['Room'].isin(exclude)]
filtered_df = filtered_df.loc[:, filtered_df.columns != 'No Icon']

# Reset the index of filtered_df
filtered_df.reset_index(drop=True, inplace=True)

# Update the filtered_df with the updated Room values for the undefined rooms
filtered_df.loc[undefined_copy.index, 'Room'] = undefined_copy['Room']

room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
filtered_df['Room'].replace({3: room_classes[3], 4: room_classes[4], 5: room_classes[5], 6: room_classes[6], 7: room_classes[7]}, inplace=True)



In [13]:
df=filtered_df

In [14]:
room_model=lr

# Relabeling Undefined Rooms in Floorplans

In [15]:
import os
import sys
import pandas as pd
import pickle

# Update PYTHONPATH to include the path to the floortrans folder
os.environ['PYTHONPATH'] = '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git:' + os.environ.get('PYTHONPATH', '')

%matplotlib inline
from skimage import transform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from floortrans.models import get_model
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns
from floortrans.plotting import segmentation_plot, polygons_to_image, draw_junction_from_dict, discrete_cmap
from floortrans.plotting import discrete_cmap
from floortrans.post_prosessing import split_prediction, get_polygons, split_validation
from mpl_toolkits.axes_grid1 import AxesGrid
import cv2
import random
from shapely.geometry import Polygon

rot = RotateNTurns() #
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]

data_folder = '../data/cubicasa5k/'
##### MODIFY THIS WHEN YOU RUN
data_file = 'train.txt' 
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)

# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]

print("Model loaded.")

Model loaded.


In [17]:
def isolate_class(rooms, CLASS: int):
    template = np.zeros_like(rooms)
    rows, cols = np.where(rooms == CLASS)
    template[rows, cols] = 1
    return template

def relabel_class(inp, contours, original_class: int, target_class: int):
    cv2.fillPoly(inp, pts=[contours], color=target_class)
    return inp

normal_set_copy=list(normal_set)
print(done)
for idx, val in enumerate(normal_set):
    print(idx)
    label = val['label']
    label_np = label.data.numpy()
    rooms=label_np[0].astype(np.uint8)
    icons=label_np[1].astype(np.uint8)
    modified_rooms=rooms.copy()
    
    
    #step 1 isolate rooms
    isolated=isolate_class(modified_rooms, 11)
    
    contours, _ = cv2.findContours(isolated.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # find individual incidents of the class, RETR_EXTERNAL just means only consider external contours (no donuts)        
   
    #step 2 look at each undefined room
    
    for rc in contours:
        rc=np.squeeze(rc,1) # removes the first dimension
        if rc.shape[0]>=4:
            
            room_cont = Polygon(rc).buffer(0) # the .buffer(0) operation is the simpliest way to ensure that the polygon stays valid
            x = np.zeros(len(icon_classes)) # initiate a new x incident
            for icon in range(len(icon_classes)): # loop through all the icons
                isolated_icon = isolate_class(icons, icon) # selects the region on the image where the class exists
                icon_contours, _ = cv2.findContours(isolated_icon, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # find indivudual incidents of the class
                for ic in icon_contours: # loop through each individual incidents
                    ic = np.squeeze(ic, 1) # removes the first dimension
                    if ic.shape[0]>=4:
                        #instead can take the average of the icon. and see if that point 
                        icon_cont = Polygon(ic).buffer(0) # again...
                        if room_cont.intersects(icon_cont): # <- if they intersects then theres one incident of overlapping between the room type and icon type
                            x[icon] += 1 # for this room type incident, there's one more incident of overlap with this icon class
            
            #step 3: predict the room
            probabilities = room_model.predict_proba([x[1:]])[0]
            probability = max(probabilities)
            prediction = np.argmax(probabilities)
            classes=room_model.classes_
            prediction=classes[prediction]
            prediction=room_classes[int(prediction)]
            if probability<0.5:
                prediction="Undefined"
            if x[6]+x[7]>0: #sauna or toilet then bath
                prediction="Bath"
            if sum(x[1:])==0:
                prediction="Undefined"
            
            #step 4: relabel class
            modified_rooms=relabel_class(modified_rooms, rc, 11, room_classes.index(prediction))
    val['modified_floorplan']=modified_rooms
    normal_set_copy[idx]=val
    # Save the modified normal_set_copy to a file
    with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/modified_floorplans/train.pkl', 'wb') as file:
        pickle.dump(normal_set_copy, file)


KeyboardInterrupt: 

In [ ]:
open_file = open("/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/modified_floorplans/train.pkl", "wb")
pickle.dump(normal_set_copy, open_file)
open_file.close()